In [1]:
#Googleドライブにアクセスできるようにする
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/UmaMusume_detection

/content/drive/MyDrive/UmaMusume_detection


In [3]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset

path = "train_data_augment/"

#Transformを作成する。
train_transform = transforms.Compose([transforms.ToTensor()])

# ImageFolderのインスタンス生成
dataset = ImageFolder(root=path, # 画像が保存されているフォルダのパス
                       transform=train_transform) # Tensorへの変換

print(dataset.class_to_idx)

# DataLoader を作成する。
dataloader = DataLoader(dataset, batch_size=64)

{'elcondorpasa': 0, 'grasswonder': 1, 'kinghalo': 2, 'seiunsky': 3, 'specialweek': 4}
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64, 3, 64, 64]) torch.Size([64])
torch.Size([64

KeyboardInterrupt: ignored

In [ ]:
#ウマ娘の名前のリスト
names = ["specialweek", "seiunsky", "kinghalo", "elcondorpasa", "grasswonder"]

#教師データのラベル付け
X_train = []
y_train = []

for name in names:
  

In [ ]:
#必要なライブラリの読み込み
import torch.nn as nn
import torch.nn.functional as F

#モデルの定義
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.cn1 = nn.Conv2d(3, 6, 5)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.cn2 = nn.Conv2d(6, 16, 5)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.cn3 = nn.Conv2d(16, 32, 5)
    self.dropout = nn.Dropout()
    self.fc1 = nn.Linear(32*10*10, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 5)
  
  def forward(self, x):
    x = F.relu(self.cn1(x))
    x = self.pool1(x)
    x = F.relu(self.cn2(x))
    x = self.pool2(x)
    x = F.relu(self.cn3(x))
    x = self.dropout(x)
    x = x.view(-1, 32*10*10)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
        
    return x